# Single-View Geometry (Python)

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- You will need [GUI backend](https://matplotlib.org/faq/usage_faq.html#what-is-a-backend) to enable interactive plots in `matplotlib`.
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

from PIL import Image
import pickle
from sympy import Symbol, Matrix, solve
from sympy import *

# Provided functions

In [2]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.imshow(im)
    print('Set at least {} lines to compute vanishing point'.format(min_lines))
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least {} lines, you have {} now'.format(min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [3]:
def plot_lines_and_vp(ax, im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10
    
    ax.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        ax.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    ax.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    ax.set_xlim([bx1, bx2])
    ax.set_ylim([by2, by1])

# Your implementation

In [4]:
def get_vanishing_point(lines):
    """
    Solves for the vanishing point using the user-input lines.
    """
    # <YOUR IMPLEMENTATION>
    AtA = lines.dot(lines.T)
    w, v = np.linalg.eig(AtA)
    # Find min eigenvalue and eigen vec.
    min_idx = np.argmin(w)
    vpts = v[:, min_idx]
    # Convert to homogeneous.
    vpts = vpts / vpts[-1]
    return vpts

In [5]:
def get_horizon_line(vpts):
    """
    Calculates the ground horizon line.
    """
    a, b, c = np.cross(vpts[:, 0], vpts[:, 1])
    scale = np.sqrt(1 / (a**2 + b**2))
    line = np.array([a, b, c]) * scale
    
    print(line)
    return line

In [6]:
def plot_horizon_line(img, h_line):
    """
    Plots the horizon line.
    """
    A = -h_line[0] / h_line[1]
    B = -h_line[2] / h_line[1]
    x = np.linspace(0,img.shape[1])
    y = A * x + B
    plt.imshow(img)
    plt.plot(x, y, 'r')
    plt.show()

In [7]:
def get_camera_parameters(vpts):
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    """
    pt1, pt2, pt3 = Matrix(vpts[:, 0]), Matrix(vpts[:, 1]), Matrix(vpts[:, 2])
    f, px, py = Symbol('f'), Symbol('px'), Symbol('py')
    K = Matrix( ((f, 0, px), 
                 (0, f, py), 
                 (0, 0, 1)) )
    K_inv = K.inv()
    Eq1 = pt1.T * K_inv.T * K_inv * pt2
    Eq2 = pt1.T * K_inv.T * K_inv * pt3
    Eq3 = pt2.T * K_inv.T * K_inv * pt3
    res = solve([Eq1, Eq2, Eq3], [f, px, py])
#     print(res)
    f, px, py = res[0]
    K = np.array([
        [f, 0, px],
        [0, f, py], 
        [0, 0, 1]
    ])
    return abs(f), px, py, K

In [8]:
def get_rotation_matrix(vpts, K):
    """
    Computes the rotation matrix using the camera parameters.
    """
    vp1 = vpts[:, 1].T
    vp2 = vpts[:, 2].T
    vp3 = vpts[:, 0].T
    K_f=np.matrix(K, dtype='float')
    K_inv = np.linalg.inv(K_f)
    
    r1 = np.dot(K_inv, vp1)
    r2 = np.dot(K_inv, vp2)
    r3 = np.dot(K_inv, vp3)
    
    r1 =  r1 / np.linalg.norm(r1)
    r2 =  r2 / np.linalg.norm(r2)
    r3 =  r3 / np.linalg.norm(r3)
    R = np.vstack((r1, r2, r3)).T
    
    return R

In [9]:
def get_homography(Rt, K):
    """
    Compute homography for transforming the image into fronto-parallel 
    views along the different axes.
    """
    H = np.zeros((3, 3, 3))
    K_f=np.matrix(K, dtype='float')
    K_inv = np.linalg.inv(K_f)
    for i in range(3):
        H[i] = np.dot(K, Rt[i]).dot(K_inv)
    return H

In [10]:
def get_rotation_matrix_rectification(R):
    """
    Compute the rotation matrix that will be used to compute the 
    homography for rectification.
    """
    R = np.array(R)
    Rt = np.zeros((3,3,3))
    XYZ = np.array([[-1., 0., 0.],
                    [0., -1., 0.],
                    [0., 0., 1.]])
    YZX = np.array([[0., 0., 1.],
                    [0., -1., 0.],
                    [1., 0., 0.]])
    ZXY = np.array([[1., 0., 0.],
                    [0., 0., -1.],
                    [0., -1., 0.]])
    
    R_inv = np.linalg.inv(R)
    
    Rt[0] = XYZ.dot(R_inv)
    Rt[1] = YZX.dot(R_inv)
    Rt[2] = ZXY.dot(R_inv)
    return Rt

# Main function

In [11]:
im = np.asarray(Image.open('./data/Q3/eceb.png'))
num_vpts = 3
# Also loads the vanishing line data if it exists in data.pickle file. 
# data.pickle is written using snippet in the next cell.
if os.path.exists('data.pickle'):
    with open('data.pickle', 'rb') as f:
        all_n, all_lines, all_centers = pickle.load(f, encoding='latin')

In [12]:
# Click and save the line data for vanishing points. This snippet 
# opens up an interface for selecting points and writes them to 
# data.pickle file. The file is over-written.

# all_n, all_lines, all_centers = [], [], []

# for i in range(num_vpts):
#     print('Getting vanishing point {}'.format(i))
#     fig = plt.figure(); ax = fig.gca()
    
#     # Get at least three lines from user input
#     n_i, lines_i, centers_i = get_input_lines(im)
#     all_n.append(n_i)
#     all_lines.append(lines_i)
#     all_centers.append(centers_i)

# with open('data.pickle', 'wb') as f:
#     pickle.dump([all_n, all_lines, all_centers], f)

In [13]:
# Part (a)
# Computing vanishing points for each of the directions
vpts = np.zeros((3, num_vpts))

for i in range(num_vpts):
    fig = plt.figure(); ax = fig.gca()
    
    # <YOUR CODE> Solve for vanishing point
    vpts[:, i] = get_vanishing_point(all_lines[i])
    
    # Plot the lines and the vanishing point
    plot_lines_and_vp(ax, im, all_lines[i], vpts[:, i])
    fig.savefig('Q3_vp{:d}.pdf'.format(i), bbox_inches='tight')
print(vpts)

[[6.82509355e+02 6.16417770e+03 1.00151052e+03]
 [2.31005292e+02 1.69245604e+02 3.53815386e+03]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00]]


In [14]:
# Part (b) Computing and plotting the horizon
# <YOUR CODE> Get the ground horizon line
horizon_line = get_horizon_line(vpts)

# <YOUR CODE> Plot the ground horizon line
fig = plt.figure(); 
ax = fig.gca()
plot_horizon_line(im, horizon_line)
fig.savefig('Q3_horizon.pdf', bbox_inches='tight')

[ 1.12658710e-02  9.99936538e-01 -2.38679694e+02]


In [15]:
# Part (c) Computing Camera Parameters
# <YOUR CODE> Solve for the camera parameters (f, u, v)
f, u, v, K = get_camera_parameters(vpts)
print(u, v, f, K)

969.200081443265 670.343110560968 1126.58813270604 [[-1126.58813270604 0 969.200081443265]
 [0 -1126.58813270604 670.343110560968]
 [0 0 1]]


In [16]:
# Part (d) Computing Rotation Matrices
# <YOUR CODE> Solve for the rotation matrix
R = get_rotation_matrix(vpts, K)
print(R)

[[-0.97297035 -0.01048588  0.23069189]
 [ 0.09385084 -0.93070604  0.35352267]
 [ 0.21099934  0.36561771  0.90653349]]


In [17]:
# Part (e) Generating fronto-parallel warps. Compute the 
# appropriate rotation to transform the world coordinates
# such that the axis of projection becomes the world
# X, Y and Z axes respectively. Use this rotation to estimate
# a homography that will be used to compute the output view.
# Apply the homography to generate the 3 fronto-parallel
# views and save them.

Rt = get_rotation_matrix_rectification(R)
print(Rt)
H = get_homography(Rt, K)
print(H)

im_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
for i in range(3):
    Ht = 0
    if i == 0: #Z
        Ht = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]) # translate
    elif i == 1: #Y
        Ht = np.array([[1, 0, 3000], [0, 1, 1000], [0, 0, 1]]) # translate
    else: #X
        Ht = np.array([[1, 0, 0], [0, 1, 3000], [0, 0, 1]]) # translate
    result = cv2.warpPerspective(im_rgb, Ht.dot(H[i]), (5000,4000))
    # Ht = np.array([[1, 0, t[0]], [0, 1, t[1]], [0, 0, 1]]) # translate
    # result = cv2.warpPerspective(img2, Ht.dot(H), (xmax-xmin, ymax-ymin))    # (h, w)
    print(i)
    # cv2.imshow('123',result)
#     fig = plt.figure(); ax = fig.gca()
    cv2.imwrite('Q3_im{:d}.jpg'.format(i), result,
                [int(cv2.IMWRITE_JPEG_QUALITY), 90])

[[[ 0.97297035 -0.09385084 -0.21099934]
  [ 0.01048588  0.93070604 -0.36561771]
  [ 0.23069189  0.35352267  0.90653349]]

 [[ 0.23069189  0.35352267  0.90653349]
  [ 0.01048588  0.93070604 -0.36561771]
  [-0.97297035  0.09385084  0.21099934]]

 [[-0.97297035  0.09385084  0.21099934]
  [-0.23069189 -0.35352267 -0.90653349]
  [ 0.01048588  0.93070604 -0.36561771]]]
[[[ 7.74506871e-01 -3.97985238e-01  6.32456227e+02]
  [-1.26780540e-01  7.20352783e-01  6.59581233e+02]
  [-2.04770389e-04 -3.13799393e-04  1.31535023e+00]]

 [[ 1.06773509e+00  2.72783101e-01 -2.03449651e+03]
  [ 5.89423249e-01  8.74862863e-01 -6.04384833e+02]
  [ 8.63643349e-04 -8.33053693e-05 -5.70200684e-01]]

 [[-9.81991316e-01 -7.06832521e-01  8.33500313e+02]
  [-2.36931204e-01 -9.07311911e-01  1.61404460e+03]
  [-9.30764261e-06 -8.26128038e-04  1.97192502e-01]]]
0
1
2
